In [251]:
# Import tools Libraries
import itertools
import re
import pandas as pd
import numpy as np

# Import Graphing Libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

# Import Text Analysis Libraries
import textblob
from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")

# Import Custom Functions
from Functions import eval_subjectivity
from Functions import eval_polarity

In [169]:
# results_table = pd.read_csv('Project_Dataset/results_table.csv')
# fresh_results_data = pd.read_csv('Project_Dataset/results.csv')

# # Combine multiple accumulated result Datasets 
# results_table = results_data.append(fresh_results_data, ignore_index=True, sort=True)
# # Reorder columns
# results_table = results_table[['title', 'source', 'label']]
# # Sort by Source & remove rows with duplicate titles
# results_table.sort_values("source",ignore_index=True).drop_duplicates(subset ="title", keep = 'first', inplace = True) 
# results_table.to_csv(path_or_buf='Project_Dataset/results_table_.csv', index=False)

In [254]:
results_table = pd.read_csv('Project_Dataset/results_table.csv')

In [256]:
subjectivity = []
polarity = []
for headline in results_table['title']:
    title = TextBlob(headline)
    subjectivity.append(title.subjectivity)
    polarity.append(title.polarity)

results_table['Subjectivity'] = [eval_subjectivity(row) for row in subjectivity]
results_table['Polarity'] = [eval_polarity(row) for row in polarity]
results_table

,title,source,label,Subjectivity,Polarity
0,'Devastating' New Jersey wildfire intentionall...,ABC News,1,Subjective,Negative
1,Biden 'just fine' after tripping 3 times joggi...,ABC News,0,Neutral,Positive
2,Republican U.S. Rep. Tom Reed accused of sexua...,ABC News,0,Subjective,Positive
3,Mexico: 2.7M U.S. vaccine doses to arrive next...,ABC News,1,Objective,Neutral
4,Report: DOJ investigating Visa over debit card...,ABC News,1,Objective,Neutral
...,...,...,...,...,...
239,"Trump unfit to be U.S. president, Democrat Pel...",Reuters,0,Objective,Neutral
240,Tesla wants to start building a new U.S. vehic...,Reuters,0,Objective,Neutral
241,Trump pledges new list of conservative Supreme...,Reuters,0,Neutral,Neutral
242,U.S. Supreme Court blocks Trump bid to end 'Dr...,Reuters,0,Objective,Neutral


In [271]:
# Dictionize required data for each unique news source
data = []
for i in results_table['source'].unique():
    data.append(
    {"Source": i, 
     "Total": len(results_table.loc[(results_table['source'] == i)]),
     "Real": len(results_table.loc[(results_table['source'] == i) & (results_table['label'] == 1)]),
     "Fake": len(results_table.loc[(results_table['source'] == i) & (results_table['label'] == 0)]),
     "# Neutral Articles": len(results_table.loc[(results_table['source'] == i) & (results_table['Subjectivity'] == "Neutral")]),
     "# Subjective Articles": len(results_table.loc[(results_table['source'] == i) & (results_table['Subjectivity'] == "Subjective")]),
     "# Objective Articles": len(results_table.loc[(results_table['source'] == i) & (results_table['Subjectivity'] == "Objective")]),
     "# Positive Headlines": len(results_table.loc[(results_table['source'] == i) & (results_table['Polarity'] == "Positive")]),
     "# Negative Headlines": len(results_table.loc[(results_table['source'] == i) & (results_table['Polarity'] == "Negative")]),
     "# Neutral Headlines": len(results_table.loc[(results_table['source'] == i) & (results_table['Polarity'] == "Neutral")])
    })

# Convert to Dataframe format 
pd.set_option('display.max_colwidth', None)
data = pd.DataFrame(data)
data['Reliability - Real'] = data.apply(lambda row: (row.Real/row.Total)*100, axis=1)
data['Reliability - Fake'] = data.apply(lambda row: 100-row["Reliability - Real"], axis=1)
data.to_excel('Project_Dataset/Compiled Results.xlsx',sheet_name='Compiled Results', index=False)
data

,Source,Total,Real,Fake,# Neutral Articles,# Subjective Articles,# Objective Articles,# Positive Headlines,# Negative Headlines,# Neutral Headlines,Reliability - Real,Reliability - Fake
0,ABC News,40,33,7,4,6,30,6,4,30,82.5,17.5
1,BBC News,40,38,2,3,4,33,3,6,31,95.0,5.0
2,Breitbart News,40,14,26,3,7,30,4,4,32,35.0,65.0
3,CNN,40,25,15,10,5,25,3,2,35,62.5,37.5
4,Fox News,40,34,6,10,6,24,5,9,26,85.0,15.0
5,Reuters,40,20,20,16,0,24,4,4,32,50.0,50.0
6,The Washington Post,4,4,0,0,0,4,0,0,4,100.0,0.0


In [ ]:
# Graph Results